In [ ]:
import glob
import os
import json

In [ ]:
# import pandas as pd

# data = pd.DataFrame()

data = []

path = 'medical-records/HEM/HEM1_merged.json'
with open(path, 'r') as file:
        try:
            text = json.load(file)
            data.append(text)
        except json.JSONDecodeError:
            print(f"Error decoding JSON in file: {path}")

path = 'medical-records/HEM/HEM2_merged.json'
with open(path, 'r') as file:
        try:
            text = json.load(file)
            data.append(text)
        except json.JSONDecodeError:
            print(f"Error decoding JSON in file: {path}")

In [ ]:
len(data)

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

In [ ]:
# load_dotenv(override=True)
# openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
model = 'qwen2.5:3b'
base_url = 'http://localhost:11434/v1'
client = OpenAI(base_url=base_url, api_key='ollama')

In [ ]:
!ollama pull qwen2.5:3b

In [ ]:
response = client.chat.completions.create(model=model, messages=[{"role": "user", "content": "hello?"}])
print(response.choices[0].message.content)

In [ ]:
system_message = f"You are a doctor who answer questions contained in medical records. \n"
system_message = f"Note that the dates recorded in the medical records are usually in Buddhist calendar in the format of DD/MM/YYYY.\n"
system_message = f"Otherwise, the dates is in Christian calendar format. \n"
system_message += f"Here is a list of medical records. There are {len(data)} records. They are in json format: \n\n"
system_message += f"{data} \n\n"
system_message += "Give your answer precisely based on the given medical records.\n"
system_message += "If you do not know the answer, please say so."

def user_prompt(message):
    return {"role": "user", "content": message}

def system_prompt():
    return {"role": "system", "content": system_message}

In [ ]:
def chat(message, history):
    messages = [system_prompt()] + history + [user_prompt(message=message)]
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type='messages').launch()

In [ ]:
base_path = 'medical-records/HEM/*'
file_paths = glob.glob(base_path)

In [ ]:
len(file_paths)

In [ ]:
merged_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            merged_data.append(data)
        except json.JSONDecodeError:
            print(f"Error decoding JSON in file: {file_path}")

output_file_path = 'merged_file.json'
with open(output_file_path, 'w') as outfile:
    json.dump(merged_data, outfile, indent=4)

print(f"Merged data written to: {output_file_path}")